#### Topics:

    Variables

A variable is an object that holds a single value of a specific type e.g., integer, date, or varying character string.

We typically use variables in the following cases:

    As a loop counter to count the number of times a loop is performed.
    To hold a value to be tested by a control-of-flow statement such as WHILE.
    To store the value returned by a stored procedure or a function

In [2]:
import pyodbc
import os
import pandas as pd

#Check if drivers are installed
#[x for x in pyodbc.drivers() if x.startswith("Microsoft Access Driver")]

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=BikeStores;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Create a cursor
cursor = conn.cursor()

#### 1. Declaring a variable

In [5]:
cursor.execute('''
DECLARE @model_year SMALLINT;
''')

cursor.execute('''
DECLARE @model_year AS SMALLINT;
''')

#Declaring multiple variables
cursor.execute('''
DECLARE @model_year SMALLINT, 
        @product_name VARCHAR(MAX);
''')


In [6]:

#Now, you can put everything together and execute the following code block to get
#a list of products whose model year is 2018:

cursor.execute('''
DECLARE @model_year SMALLINT;

SET @model_year = 2018;

SELECT
    product_name,
    model_year,
    list_price 
FROM 
    production.products
WHERE 
    model_year = @model_year
ORDER BY
    product_name;
''')



# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,model_year,list_price
0,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,2018,899.99
1,Electra Amsterdam Royal 8i - 2017/2018,2018,1259.90
2,Electra Amsterdam Royal 8i Ladies - 2018,2018,1199.99
3,Electra Cruiser 1 - 2016/2017/2018,2018,269.99
4,Electra Cruiser 1 Ladies' - 2018,2018,269.99
5,Electra Cruiser 1 Tall - 2016/2018,2018,269.99
6,Electra Cruiser 7D (24-Inch) Ladies' - 2016/2018,2018,319.99
7,Electra Cruiser 7D (24-Inch) Ladies' - 2016/2018,2018,319.99
8,Electra Cruiser 7D - 2016/2017/2018,2018,319.99
9,Electra Cruiser 7D Ladies' - 2016/2018,2018,319.99


#### 2. Storing query result in a variable

In [7]:
cursor.execute('''
DECLARE @product_count INT;

SET @product_count = (
    SELECT 
        COUNT(*) 
    FROM 
        production.products 
);

SELECT @product_count;

''')



# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,
0,321


#### 3. Selecting a record into variables

In [11]:
cursor.execute('''
DECLARE 
    @product_name VARCHAR(MAX),
    @list_price DECIMAL(10,2);

SELECT 
    @product_name = product_name,
    @list_price = list_price
FROM
    production.products
WHERE
    product_id = 100;

SELECT 
    @product_name AS product_name, 
    @list_price AS list_price;
''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Electra Townie 3i EQ (20-inch) - Boys' - 2017,489.99


In [12]:
conn.close()